In [1]:
import requests
from pathlib import Path

def download_one_file_of_raw_data(year: int, month: int) -> Path:
    """"""
    URL = f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year}-{month:02d}.parquet'
    response = requests.get(URL)

    if response.status_code == 200:
        path = f'../data/raw/yellow_tripdata_{year}-{month:02d}.parquet'
        open(path, 'wb').write(response.content)
        return Path(path)
    else:
        raise Exception(f'Failed to download {URL}')

In [2]:
download_one_file_of_raw_data(2023, 6)

WindowsPath('../data/raw/yellow_tripdata_2023-06.parquet')

In [3]:
import pandas as pd

rides = pd.read_parquet('../data/raw/yellow_tripdata_2023-06.parquet')
rides.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2023-06-01 00:08:48,2023-06-01 00:29:41,1.0,3.40,1.0,N,140,238,1,21.9,3.50,0.5,6.70,0.0,1.0,33.60,2.5,0.00
1,1,2023-06-01 00:15:04,2023-06-01 00:25:18,0.0,3.40,1.0,N,50,151,1,15.6,3.50,0.5,3.00,0.0,1.0,23.60,2.5,0.00
2,1,2023-06-01 00:48:24,2023-06-01 01:07:07,1.0,10.20,1.0,N,138,97,1,40.8,7.75,0.5,10.00,0.0,1.0,60.05,0.0,1.75
3,2,2023-06-01 00:54:03,2023-06-01 01:17:29,3.0,9.83,1.0,N,100,244,1,39.4,1.00,0.5,8.88,0.0,1.0,53.28,2.5,0.00
4,2,2023-06-01 00:18:44,2023-06-01 00:27:18,1.0,1.17,1.0,N,137,234,1,9.3,1.00,0.5,0.72,0.0,1.0,15.02,2.5,0.00


In [4]:
rides = rides[['tpep_pickup_datetime', 'PULocationID']]
rides.rename(columns={'tpep_pickup_datetime': 'pickup_datetime', 'PULocationID': 'pickup_location_id'}, inplace=True)

In [5]:
rides

,pickup_datetime,pickup_location_id
0,2023-06-01 00:08:48,140
1,2023-06-01 00:15:04,50
2,2023-06-01 00:48:24,138
3,2023-06-01 00:54:03,100
4,2023-06-01 00:18:44,137
...,...,...
3307229,2023-06-30 23:30:21,42
3307230,2023-06-30 23:34:22,132
3307231,2023-06-30 23:45:00,256
3307232,2023-06-30 23:13:38,91


In [6]:
rides['pickup_datetime'].describe()

C:\Users\Julian Sotelo\AppData\Local\Temp\ipykernel_14912\3389344848.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  rides['pickup_datetime'].describe()


count                 3307234
unique                1659290
top       2023-06-06 17:40:32
freq                       11
first     2002-12-31 23:03:19
last      2023-07-01 00:42:13
Name: pickup_datetime, dtype: object

In [7]:
rides = rides[rides['pickup_datetime'] < '2023-07-01']
rides = rides[rides['pickup_datetime'] >= '2023-06-01']
rides['pickup_datetime'].describe()

C:\Users\Julian Sotelo\AppData\Local\Temp\ipykernel_14912\3904695853.py:3: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  rides['pickup_datetime'].describe()


count                 3307193
unique                1659250
top       2023-06-16 15:00:16
freq                       11
first     2023-06-01 00:00:00
last      2023-06-30 23:59:59
Name: pickup_datetime, dtype: object

In [8]:
rides.to_parquet("../data/transformed/validated_rides_2023-06.parquet")